In [11]:
import requests

def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+str(appid), params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()


In [91]:
import copy
def get_n_reviews(appid, n=100):
    print(n)
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']

        if len(response['reviews']) < 100: break
    

    return reviews

In [96]:

def fill_data(appid, total_reviews):
    df_list = []
    reviews = get_n_reviews(appid, total_reviews)
    for r in reviews:
        df_dict = {}
        df_dict['ID'] = appid        
        try:
            df_dict['Recommendation ID'] = r['recommendationid']
        except: 
            df_dict['Recommendation ID'] = "error"

        try:
            df_dict['User ID'] = r['author']['steamid']
        except:
            df_dict['User ID'] = "error"

        try:
            df_dict['Num Game Owned'] = r['author']['num_games_owned']
        except:
            df_dict['Num Game Owned'] = "error"

        try:
            df_dict['Num Reviews'] = r['author']['num_reviews']
        except:
            df_dict['Num Reviews'] = "error"

        try:
            df_dict['Playtime Forever'] = r['author']['playtime_forever']
        except:
            df_dict['Playtime Forever'] = "error"

        try:
            df_dict['Playtime Last Two Weeks'] = r['author']['playtime_last_two_weeks']
        except:
            df_dict['Playtime Last Two Weeks'] = "error"

        try:
            df_dict['Playtime At Review'] = r['author']['playtime_at_review']
        except:
            df_dict['Playtime At Review'] = "error"

        try:
            df_dict['Last Played'] = r['author']['last_played']
        except:
            df_dict['Last Played'] = "error"

        try:
            df_dict['Steam Purchase'] = r['steam_purchase']
        except:
            df_dict['Steam Purchase'] = "error"

        try:
            df_dict['Received For Free'] = r['received_for_free']
        except:
            df_dict['Received For Free'] = "error"

        try:
            df_dict['Review'] = r['review']
        except:
            df_dict['Review'] = "error"

        try:
            df_dict['Time'] = r['timestamp_updated']
        except:
            df_dict['Time'] = "error"
            
        try:
            df_dict['Voted Up'] = r['voted_up']
        except:
            df_dict['Voted Up'] = "error"
        df_list.append(df_dict)
    return df_list
            

In [84]:
import pandas as pd
import time



df1 = pd.read_csv("./Game_Info.csv")
CSV_FILE = "./Game_Reviews.csv"
# with open(CSV_FILE, "w", encoding='utf-8') as f:
#     f.write("id,recommendationid,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,steam_purchase,received_for_free,review,timestamp_updated,voted_up\n")

while True:
    try:
        for appid in df1['ID'].tolist():
            if appid in existed:
                continue
            total_reviews = df1[df1['ID'] == appid]['Total Reviews'].values[0]
            if total_reviews == 0:
                continue
            df_list = fill_data(total_reviews)
            if len(df_list) != total_reviews:
                df_list2 = fill_data(total_reviews)

                if len(df_list2) > len(df_list):
                    df_list = df_list2
            
            df = pd.DataFrame.from_dict(df_list)
            df.to_csv(CSV_FILE, mode = 'a', header = False, index=False)
            if total_reviews > 1000:
                time.sleep(60)
            
            print("%d/%d: Game %d: %d reviews fetched. Total: %d" % (df1['ID'].tolist().index(appid), len(df1['ID'].tolist()), appid, total_reviews, len(df)))
            existed.append(appid)
            
        break
    except TimeoutError:
        print('Timeout Error')
        continue
    except ConnectionResetError:
        print('ConnectionResetError')
        continue
    # except ProtocolError:
    #     continue
    except ConnectionError:
        print('ConnectionError')
        continue
    
    except KeyboardInterrupt:
        print('Interrupted')
        break




5280/9235: Game 1152950: 18 reviews fetched. Total: 18
5281/9235: Game 1105590: 90 reviews fetched. Total: 90
5282/9235: Game 682530: 371 reviews fetched. Total: 371
5283/9235: Game 1067540: 550 reviews fetched. Total: 550
5284/9235: Game 897030: 729 reviews fetched. Total: 729
5285/9235: Game 1103100: 131 reviews fetched. Total: 131
5286/9235: Game 39200: 920 reviews fetched. Total: 920
5287/9235: Game 273600: 139 reviews fetched. Total: 139
5288/9235: Game 1619970: 1 reviews fetched. Total: 1
5289/9235: Game 1162130: 424 reviews fetched. Total: 424
5290/9235: Game 823940: 313 reviews fetched. Total: 313
5291/9235: Game 1399840: 95 reviews fetched. Total: 95
5292/9235: Game 1238980: 3 reviews fetched. Total: 3
5293/9235: Game 765900: 138 reviews fetched. Total: 138
5294/9235: Game 746930: 858 reviews fetched. Total: 858
5295/9235: Game 848030: 178 reviews fetched. Total: 178
5296/9235: Game 1126580: 44 reviews fetched. Total: 44
5297/9235: Game 415420: 398 reviews fetched. Total: 398


In [87]:
df1[df1['ID'] == 1245620]

,Unnamed: 0,Link,ID,Name,Price,Tag,Description,All Reviews,Recent Reviews,Release Date,Positive Rate,Developer,Num Reviews,Review Score,Review Score Desc,Total Positive,Total Negative,Total Reviews
1,1,https://store.steampowered.com/app/1245620/,1245620,ELDEN RING,$79.99,"Souls-like,Relaxing,Dark Fantasy,RPG,Difficult...","THE NEW FANTASY ACTION RPG. Rise, Tarnished, a...",Very Positive,Very Positive,"Feb 24, 2022","92% of the 130,790 user reviews in the last 30...",FromSoftware Inc.,20,8,Very Positive,158573,15479,174052


In [99]:
appid = 1245620
total_reviews = df1[df1['ID'] == appid]['Total Reviews'].values[0]
df_list = fill_data(appid, total_reviews)
if len(df_list) != total_reviews:
    df_list2 = fill_data(appid, total_reviews)

    if len(df_list2) > len(df_list):
        df_list = df_list2

df = pd.DataFrame.from_dict(df_list)
df.to_csv(CSV_FILE, mode = 'a', header = False, index=False)
print("%d reviews fetched. Total: %d" % (total_reviews, len(df)))

174052
174052 reviews fetched. Total: 174052


In [ ]:
game_reviews = pd.read_csv("./Game_Reviews.csv")
game_reviews.head()

In [ ]:
for appid in df1['ID'].tolist():
    if appid in existed:
        continue
    total_reviews = df1[df1['ID'] == appid]['Total Reviews'].values[0]
    if total_reviews == 0:
        continue
    df_list = fill_data(total_reviews)
    if len(df_list) != total_reviews:
        df_list2 = fill_data(total_reviews)

        if len(df_list2) > len(df_list):
            df_list = df_list2
    
    df = pd.DataFrame.from_dict(df_list)
    df.to_csv(CSV_FILE, mode = 'a', header = False, index=False)
    if total_reviews > 1000:
        time.sleep(60)
    
    print("%d/%d: Game %d: %d reviews fetched. Total: %d" % (df1['ID'].tolist().index(appid), len(df1['ID'].tolist()), appid, total_reviews, len(df)))
    existed.append(appid)
            